In [1]:
!pip install Pyfhel
!pip install pynacl
!pip install cryptography
!pip install tqdm

In [2]:
no_clients = 3
epochs = 3

In [3]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

# Check if GPU is available
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    print("GPUs available:", len(gpus))
    for gpu in gpus:
        print(gpu)
else:
    print("No GPU available.")

2024-06-17 19:18:16.893860: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 19:18:16.893892: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 19:18:16.893925: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-17 19:18:16.901181: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-17 19:18:17.660698: W tensorflow/compiler/

TensorFlow version: 2.14.1
GPUs available: 1
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2024-06-17 19:18:18.684850: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-17 19:18:18.705157: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-17 19:18:18.705375: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
import tensorflow as tf
from tqdm import tqdm
import copy
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import pickle
import sys
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import os
import tensorflow as tf
from src.models.FMLEE import FMLEE
from src.data.load_data import load_mnist

In [5]:


save_dir = "dataset/mnist_data/"
os.makedirs(save_dir, exist_ok=True)

In [6]:
fml = FMLEE(no_clients, epochs)

Client 0 initialized.
Client 1 initialized.
Client 2 initialized.


2024-06-17 19:18:19.586916: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-17 19:18:19.587135: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-17 19:18:19.587288: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [7]:
fml.clients

In [8]:
fml.HE

<ckks Pyfhel obj at 0x7f9895d9b7b0, [pk:Y, sk:Y, rtk:Y, rlk:Y, contx(n=16384, t=0, sec=128, qi=[60, 30, 30, 30, 60], scale=1073741824.0, )]>

In [9]:
(x_train_all, y_train_all), (x_test, y_test)  = load_mnist()


Dataset loaded from local files at dataset/mnist_data/


In [10]:
fml.clients[0].fit(x_train_all, y_train_all)

2024-06-17 19:18:20.464159: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-06-17 19:18:20.529522: W tensorflow/compiler/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:521] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  /usr/local/cuda-11.8
  /usr/local/cuda
  /home/voy/.conda/envs/he39/lib/python3.9/site-packages/tensorflow/python/platform/../../../nvidia/cuda_nvcc
  /home/voy/.conda/envs/he39/lib/python3.9/site-packages/tensorflow/python/platform/../../../../nvidia/cuda_nvcc
  .
You can choose the search directory by setting xla_gpu_cuda_data_dir in HloModule's DebugOptions.  For most apps, setting the environment variable XLA_FLAGS=--xla_gpu_cuda_data_dir=/path/to/cuda will work.
2024-06-17 19:18:20.532359: W tensorflow/compiler/xla/service/gpu/

UnknownError: Graph execution error:

Detected at node Adam/Pow defined at (most recent call last):
  File "/home/voy/.conda/envs/he39/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/home/voy/.conda/envs/he39/lib/python3.9/runpy.py", line 87, in _run_code

  File "/home/voy/.conda/envs/he39/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/voy/.conda/envs/he39/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/voy/.conda/envs/he39/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/voy/.conda/envs/he39/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/voy/.conda/envs/he39/lib/python3.9/asyncio/base_events.py", line 601, in run_forever

  File "/home/voy/.conda/envs/he39/lib/python3.9/asyncio/base_events.py", line 1905, in _run_once

  File "/home/voy/.conda/envs/he39/lib/python3.9/asyncio/events.py